In [1]:
import nltk
import re
import pandas as pd
import numpy as np
from collections import Counter
import tqdm
import pickle as pk
from bs4 import BeautifulSoup
from collections import OrderedDict

import warnings
warnings.filterwarnings("ignore")


In [2]:
from collections import Iterable
def flatten(lis):
     for item in lis:
         if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                 yield x
         else:        
             yield item


# Using xml.sax parser to get the text data from the xml file

In [3]:
import numpy as np
import xml.sax
Text = []

class StreamHandler(xml.sax.handler.ContentHandler):
  inText = False
  lastRevision = 0
  
  def startElement(self, name, attrs):
    if name == 'text':
      self.inText = True
      self.content = ''
    if name == 'revision':
      self.lastRevision += 1

  def endElement(self, name):
    if name == 'text':
      self.inText = False
      Text.append(self.content)      
 
  def characters(self, content):
    if self.inText:
      self.content += content
    
if __name__ == '__main__':
  # use default ``xml.sax.expatreader``
  parser = xml.sax.make_parser()
  parser.setContentHandler(StreamHandler())
for idx in range(1):
    with open('509th_Composite_Group_' + str(idx) + ".xml", 'r', errors='ignore') as f:
      parser.parse(f)

# Preprocessing the text data

In [4]:
def preprocessing(text):
    text= BeautifulSoup(text, 'html.parser').get_text() #remove_html_tags_func
    text=re.sub(r'https?://\S+|www\.\S+', '', text) #remove_url_func
    text=re.sub(r'[^a-zA-Z0-9]', ' ', text) #remove puntuation
    text=re.sub(r'^\s*|\s\s*', ' ', text).strip() #remove_extra_whitespaces_func
    return text
    

In [5]:
Corpus=list(map(preprocessing, Text))

# Using Flair NER to get entities of interest

In [6]:
from flair.data import Sentence
from flair.models import SequenceTagger
tagger = SequenceTagger.load("flair/ner-english-fast")

2022-09-23 21:57:29,905 loading file C:\Users\razaa\.flair\models\ner-english-fast\4c58e7191ff952c030b82db25b3694b58800b0e722ff15427f527e1631ed6142.e13c7c4664ffe2bbfa8f1f5375bd0dced866b8c1dd7ff89a6d705518abf0a611
2022-09-23 21:57:36,619 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [7]:
revisions = []
for idx, line in enumerate(tqdm.tqdm(Corpus[0:10])):
    persons_in_revision = []
    doc = Sentence(line)
    tagger.predict(doc)
    for entity in doc.get_spans('ner'):
        if 'PER' in str(entity.labels):
            persons_in_revision.append(entity.text)
    df = pd.Series(Counter(persons_in_revision))
    df = pd.DataFrame(df)
    df['revision'] = idx
    revisions.append(df.reset_index())

names_from_revisions = pd.concat(revisions)
names_from_revisions

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [06:43<00:00, 40.32s/it]


,index,0,revision
0,Paul Tibbets,1.0,2
1,Paul W Tibbets Jr,1.0,2
2,Charles Sweeney,1.0,2
3,Paul W Tibbets,3.0,2
4,Thomas J Classen,3.0,2
...,...,...,...
45,LeMay Curtis,1.0,9
46,Mann Robert,1.0,9
47,Marx Joseph,1.0,9
48,Thomas Gordon,2.0,9


In [8]:
interested_person=list(dict.fromkeys(persons_in_revision))

# Falcon2.0 API used to get the Wikidata Qids

In [9]:
import requests
import json
import pprint
                 
wiki=[]
url="https://labs.tib.eu/falcon/falcon2/api?mode=long"
headers = {'Content-type': 'application/json'}
for name in interested_person:
    data = {"text":name }
    data_json = json.dumps(data)
    response = requests.post(url, data=data_json, headers=headers)
    response_1=str(response.content)
    wiki.append(response_1)

In [10]:
matches=(re.findall("Q+[\d]+(?:\d)?",line) for line in wiki)
matches = [m for m in matches if m]

In [11]:
Qid=list(flatten(matches))
Qid=list(OrderedDict.fromkeys(Qid))
Qid=["wd:" + item for item in Qid ]
df=pd.DataFrame()
df['wiki_Qid']=Qid

In [20]:
df.to_csv('Wiki_Qid.csv', index=False)
names_from_revisions.to_csv('NER_results.csv', index=False)